In [30]:
import pandas as pd
import plotly.express as px
import data_preprocessing.data_preprocess as dp
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist

In [2]:
# Interesting variables to be considered for policy space
policy_vars = [
    "Military: Positive",
    "European Community/Union: Positive",
    "Freedom and Human Rights",
    "Democracy",
    "Political Corruption",
    "Environmental Protection",
    "Welfare State",
    "Right-left position",
    "Planned Economy",
    "Equality: Positive"]

In [7]:
party_scaled, voter_scaled = dp.get_scaled_party_voter_data(x_var='Democracy', y_var='Welfare State',year=2021)

c:\Users\natal\Case_Study_TUM25\data_preprocessing\data_loading.py:39: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
c:\Users\natal\Case_Study_TUM25\data_preprocessing\data_loading.py:54: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_filtered = df_filtered.apply(pd.to_numeric, errors="ignore")
c:\Users\natal\Case_Study_TUM25\data_preprocessing\data_loading.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_filtered['Year'] = (pd.to_datetime(df_filtered['Date'], dayfirst=True).dt.year).astype(str)
c:\Users\natal\Case_Study_TUM25\data_preprocessing\data_

voters_2021.sav


In [8]:
party_scaled

,Country,Date,Calendar_Week,Party_Name,Democracy,Welfare State,Democracy Scaled,Welfare State Scaled,Democracy Voters_Mean,Welfare State Voters_Mean,Democracy Combined,Welfare State Combined,Label
0,Germany,26/09/2021,202109,90/Greens,5.544,19.342,1.306876,0.150875,0.221756,0.528214,0.438780,0.452746,90/Greens
1,Germany,26/09/2021,202109,LINKE,5.414,28.427,1.175452,1.429001,-0.276607,0.888032,0.013805,0.996226,LINKE
2,Germany,26/09/2021,202109,SPD,3.864,22.414,-0.391529,0.583060,-0.024614,0.135886,-0.097997,0.225321,SPD
3,Germany,26/09/2021,202109,FDP,4.785,14.222,0.539561,-0.569434,-0.046485,-0.653846,0.070724,-0.636964,FDP
4,Germany,26/09/2021,202109,CDU/CSU,2.559,10.703,-1.710825,-1.064505,0.110111,-0.400705,-0.254076,-0.533465,CDU/CSU
5,Germany,26/09/2021,202109,AfD,3.558,7.687,-0.700881,-1.488812,-0.555041,-0.524120,-0.584209,-0.717059,AfD


In [9]:
voter_scaled

,Democracy,Welfare State,who did you vote for:second vote,"do you incline towards a party, if so which one",how strongly do you incline towards this party,Party_Name,party_choice,Democracy Scaled,Welfare State Scaled,Label
0,3.000000,1.000000,4.0,4.0,3.0,SPD,0,0.485631,-2.248722,Voter
1,3.000000,1.000000,5.0,5.0,2.0,FDP,1,0.485631,-2.248722,Voter
2,2.666667,3.666667,6.0,6.0,3.0,90/Greens,2,-0.006648,0.797943,Voter
3,3.000000,4.333333,4.0,4.0,2.0,SPD,0,0.485631,1.559609,Voter
4,3.333333,2.333333,1.0,0.0,3.0,CDU/CSU,3,0.977910,-0.725390,Voter
...,...,...,...,...,...,...,...,...,...,...
2735,3.000000,2.000000,5.0,1.0,2.0,FDP,1,0.485631,-1.106223,Voter
2736,1.666667,2.666667,5.0,5.0,3.0,FDP,1,-1.483484,-0.344557,Voter
2737,3.000000,2.333333,5.0,1.0,2.0,FDP,1,0.485631,-0.725390,Voter
2738,2.333333,1.000000,4.0,4.0,3.0,SPD,0,-0.498926,-2.248722,Voter


In [11]:
# Plot without scaling
fig = px.scatter(
    pd.concat([
        voter_scaled.assign(Type="Voter", Size=5, Color="Voter"),
        party_scaled.assign(Type="Party", Size=15, Color=party_scaled["Party_Name"])
    ]),
    x="Democracy",
    y="Welfare State",
    color="Color",
    symbol="Type",
    size="Size",
    title="Unscaled Voter and Party Positions"
)
fig.update_traces(textposition="top center")
fig.show()

In [14]:
concatenated_df = pd.concat([voter_scaled, party_scaled], ignore_index=True)

fig = px.scatter(
    concatenated_df,
    x='Democracy Scaled',
    y='Welfare State Scaled',
    color='Label',
    symbol='Label')
fig.update_traces(marker=dict(size=10))
fig.update_layout(title='Scaled Voter and Party Positions')
fig.show()

In [16]:
from scipy.stats import gaussian_kde
import numpy as np

x_var = "Democracy"
y_var = "Welfare State"

# Your voter cloud (already scaled to [0,10])
x = voter_scaled[f"{x_var} Scaled"].values
y = voter_scaled[f"{y_var} Scaled"].values

# Stack for 2D
data = np.vstack([x, y])

# Build KDE — you now have a continuous density function!
kde = gaussian_kde(data, bw_method='scott')  # or 'silverman', or custom float

# Evaluate anywhere in 2D space:
density_at_5_5 = kde([5, 5])  # returns a scalar density estimate


In [17]:
def voter_density(x_input, y_input):
    """
    Continuous density function over 2D space from KDE.
    
    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space (can be scalars or arrays).
    
    Returns:
    --------
    float or np.ndarray
        Density estimate(s) at the given location(s).
    """
    xy = np.vstack([np.ravel(x_input), np.ravel(y_input)])
    density_vals = kde(xy)
    return density_vals.reshape(np.shape(x_input))


In [18]:
# Scalar evaluation
voter_density(5, 5)   # → float

# Grid evaluation (e.g. for plotting or integration)
X, Y = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 10, 100))
Z = voter_density(X, Y)  # shape (100, 100)

In [19]:
from sklearn.mixture import GaussianMixture
import numpy as np

# Step 1: Prepare your 2D data
X = voter_scaled[[f"{x_var} Scaled", f"{y_var} Scaled"]].values

# Step 2: Fit GMM — choose number of components (e.g. 4)
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=0)
gmm.fit(X)


GaussianMixture(n_components=4, random_state=0)

In [20]:
from scipy.stats import multivariate_normal

def gmm_density(x_input, y_input):
    """
    Evaluate the GMM density at given (x, y).
    
    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space.
    
    Returns:
    --------
    float or np.ndarray
        Density value(s).
    """
    x_flat = np.ravel(x_input)
    y_flat = np.ravel(y_input)
    points = np.column_stack([x_flat, y_flat])
    
    density_vals = np.zeros(len(points))
    for weight, mean, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        rv = multivariate_normal(mean=mean, cov=cov)
        density_vals += weight * rv.pdf(points)
    
    return density_vals.reshape(np.shape(x_input))


In [21]:
Xgrid, Ygrid = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 10, 100))
Z = gmm_density(Xgrid, Ygrid)


In [22]:
print("Weights:", gmm.weights_)
print("Means:\n", gmm.means_)
print("Covariances:\n", gmm.covariances_)


Weights: [0.1762098  0.43550087 0.1658385  0.22245083]
Means:
 [[-1.42467127 -0.00853157]
 [ 0.18879468 -0.0776696 ]
 [ 0.41688876 -1.26521089]
 [ 0.44811963  1.10203767]]
Covariances:
 [[[ 0.84018245  0.0440322 ]
  [ 0.0440322   1.09490141]]

 [[ 0.41041078 -0.00221388]
  [-0.00221388  0.22800006]]

 [[ 0.9705861  -0.08021978]
  [-0.08021978  0.46509448]]

 [[ 0.29485797  0.02938366]
  [ 0.02938366  0.41525035]]]


In [23]:
from scipy.stats import multivariate_normal
import numpy as np

def gmm_indefinite_integral(x, y):
    total_cdf = 0
    point = np.array([x, y])
    for w, mu, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        total_cdf += w * multivariate_normal.cdf(point, mean=mu, cov=cov)
    return total_cdf


In [24]:
def gmm_density_and_loggrad(x_input, y_input, gmm):
    """
    Evaluate the GMM density and gradient of log-density at given (x, y).

    Parameters:
    -----------
    x_input, y_input : float or np.ndarray
        Coordinates in policy space.
    gmm : fitted sklearn.mixture.GaussianMixture object

    Returns:
    --------
    density_vals : np.ndarray
        GMM density values at input points.
    grad_log_density : np.ndarray
        Gradient of log-density at input points. Shape: (N, 2)
    """
    x_flat = np.ravel(x_input)
    y_flat = np.ravel(y_input)
    points = np.column_stack([x_flat, y_flat])  # Shape: (N, 2)
    N = len(points)

    density_vals = np.zeros(N)
    grad = np.zeros_like(points)

    # Loop over components
    for weight, mean, cov in zip(gmm.weights_, gmm.means_, gmm.covariances_):
        rv = multivariate_normal(mean=mean, cov=cov)
        pdf_vals = rv.pdf(points)                          # (N,)
        diff = points - mean                               # (N, 2)
        inv_cov = np.linalg.inv(cov)                       # (2, 2)
        grad_comp = -pdf_vals[:, None] * (diff @ inv_cov.T)  # (N, 2)

        density_vals += weight * pdf_vals
        grad += weight * grad_comp

    # Compute ∇ log p(x) = ∇p(x) / p(x)
    eps = 1e-9  # to avoid divide-by-zero
    grad_log_density = grad / (density_vals[:, None] + eps)

    # Reshape density to match input shape
    density_vals = density_vals.reshape(np.shape(x_input))

    return grad_log_density


In [25]:
def run_simulation(data,T,N,D,sigma_noise,gmm_components,alpha,beta,gamma):
    history = [data.copy()]
    
    for t in range(T):
    # Current positions
        X_t = history[-1]

    # Compute weight matrix w_ij = exp(-||x_i - x_j||)
        distances = cdist(X_t.T, X_t.T,metric='euclidean')  
        W = np.exp(-distances)
        W /= W.sum(axis=1, keepdims=True)

    # Weighted average term
        weighted_sum = W @ X_t.T
        #print(weighted_sum.shape)
        
    # Drift term: -∇ log gmm_density(x_i)
        F_x=gmm_density_and_loggrad(X_t[0,:],X_t[1,:],gmm)
       

    # Error term (Gaussian noise)
        noise = np.random.normal(0, sigma_noise, size=(N, D))
        
    # Update rule
        X_next = alpha*weighted_sum*X_t.T - beta*F_x + gamma*noise

        history.append(X_next.T)
        
    final_positions = history[-1]
    return final_positions    

In [32]:
def plot_with_simulation_separate(concatenated_df, simulation_points):
    fig = px.scatter(
        concatenated_df,
        x='Democracy Scaled',
        y='Welfare State Scaled',
        color='Label',
        symbol='Label'
    )
    
    # Add simulation points as a separate trace
    fig.add_scatter(
        x=simulation_points[0],
        y=simulation_points[1],
        mode='markers',
        marker=dict(
            color='rgba(0,0,0,0.2)',
            size=4,
            symbol='circle'
        ),
        name='Simulation Points'
    )
    
    fig.update_layout(title='Scaled Positions with Simulation Overlay')
    return fig

In [28]:
N = x.__len__()             
D = 2                
T = 500               
sigma_noise = 0.05   
gmm_components = 4 

In [45]:
sim=run_simulation(data,10,N,D,sigma_noise,gmm_components,0.5,0.1,1)
plot_with_simulation_separate(concatenated_df,sim)